In [4]:
import tweepy
import pandas as pd
import time

### Twitter Credentials (Updating the twitter credentials in the twitter_credentials.csv file and loading it here)

In [5]:
credentials_df = pd.read_csv('twitter_credentials.csv',header=None,names=['Name','Key'])
credentials_df

,Name,Key
0,consumer_key,CvbMui2D4MuIK5dxzT0mR03s8
1,consumer_secret,YkrRUAfCCi9h0mDzThuzNb5d7CSDjQwb4SwDzQWaErVopt...
2,access_token,895342019668619264-lQ9NRAL8URrYfIpRSS1fbVMcUpk...
3,access_secret,naWmOWyX6XDN8f1khdmh2KodtOIr6DOmEjqipyQyetUgV


In [6]:
consumer_key = credentials_df.loc[credentials_df['Name']=='consumer_key','Key'].iloc[0]
consumer_secret = credentials_df.loc[credentials_df['Name']=='consumer_secret','Key'].iloc[0]
access_token = credentials_df.loc[credentials_df['Name']=='access_token','Key'].iloc[0]
access_token_secret = credentials_df.loc[credentials_df['Name']=='access_secret','Key'].iloc[0]

In [7]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [8]:
tweet_url = pd.read_csv("Democrat/Urls/dem_tweets_oct_urls.txt", index_col= None, header = None, names = ["links"])
tweet_url.head()

,links
0,https://twitter.com/ncc8288/status/13226898109...
1,https://twitter.com/DemocratsCare/status/13226...
2,https://twitter.com/rjschmidtco/status/1322689...
3,https://twitter.com/dino4521/status/1322689798...
4,https://twitter.com/DisneyDemocrat/status/1322...


In [9]:
af = lambda x: x["links"].split("/")[-1]
tweet_url['id'] = tweet_url.apply(af, axis=1)
tweet_url.head()

,links,id
0,https://twitter.com/ncc8288/status/13226898109...,1322689810977431554
1,https://twitter.com/DemocratsCare/status/13226...,1322689810918682633
2,https://twitter.com/rjschmidtco/status/1322689...,1322689801896796167
3,https://twitter.com/dino4521/status/1322689798...,1322689798793035776
4,https://twitter.com/DisneyDemocrat/status/1322...,1322689797866082304


In [10]:
ids = tweet_url['id'].tolist()

In [11]:
total_count = len(ids)
chunks = (total_count - 1) // 50 + 1

In [12]:
total_count

602179

In [94]:
def fetch_tw(ids):
    list_of_tw_status = api.statuses_lookup(ids, tweet_mode= "extended")
    empty_data = pd.DataFrame()
    for status in list_of_tw_status:
            tweet_elem = {"tweet_id": status.id,
                     "screen_name": status.user.screen_name,
                     "tweet":status.full_text,
                     "date":status.created_at,
                     "coordinates":status.coordinates,
                     "location": status.user.location,
                     "place": status.place
                        }
            empty_data = empty_data.append(tweet_elem, ignore_index = True)
    empty_data.to_csv("Republican/Tweets/rep_tweets_oct.csv", mode="a")

In [95]:
for i in range(chunks):
        batch = ids[i*50:(i+1)*50]
        result = fetch_tw(batch)

KeyboardInterrupt: 